# Particle vs Parametric

Synthesizer contains two types of galaxy:

- A particle `Galaxy` for working with galaxies comprised of individual star, gas, and/or black hole particles (`synthesizer.particle.galaxy.Galaxy`).
- A parametric `Galaxy` for working with parametric models described by a SFZH and (when necessary) morphologies (`synthesizer.parametric.galaxy.Galaxy`).

To simplify the instatiation of a Galaxy for the user we provide a ``Galaxy`` "factory function" which will always return the correct galaxy based on the arguments passed by the user, while raising errors or warnings for incompatible argument combinations.

Below we set up the components (particle and parametric stellar populations) we will need to demonstrate how to define different galaxies.
Further use of a galaxy for [spectra](../spectra/galaxy.ipynb), [line emission](../lines/lines.rst), [imaging](../imaging/imaging.rst) and much more can be found elsewhere in the documentation.

In [ ]:
import numpy as np
from unyt import Msun, Myr, kpc

from synthesizer.grid import Grid
from synthesizer.parametric import SFH, ZDist
from synthesizer.parametric import Stars as ParametricStars
from synthesizer.particle.stars import sample_sfzh

# Define the grid
grid_name = "test_grid"
grid_dir = "../../../tests/test_grid/"
grid = Grid(grid_name, grid_dir=grid_dir)

# Define the metallicity history
zh = ZDist.DeltaConstant(metallicity=0.01)

# Define the star formation history
sfh_p = {"duration": 100 * Myr}
sfh = SFH.Constant(100 * Myr)

# Initialise the parametric Stars object
param_stars = ParametricStars(
    grid.log10age,
    grid.metallicity,
    sf_hist=sfh,
    metal_dist=zh,
    initial_mass=10**9 * Msun,
)

# Define the number of stellar particles we want
n = 10000

# Sample the parametric SFZH, producing a particle Stars object
# we will also pass some keyword arguments for some example attributes
part_stars = sample_sfzh(
    sfzh=param_stars.sfzh,
    log10ages=param_stars.log10ages,
    log10metallicities=param_stars.log10metallicities,
    nstar=n,
    current_masses=np.full(n, 10**8.7 / n) * Msun,
    redshift=1,
    initial_mass=10**6 * Msun,
)

# Show what we've got out
print(type(param_stars))
print(type(part_stars))

## Creating a Galaxy

Now that we have the building blocks of both a particle and parametric galaxy we can import the factory function and get our galaxies. 

To do so we simply pass the factory function the arguments for the desired type of `Galaxy`. These are `Stars`, `Gas`, and `BlackHoles` objects from the `particle` and `parametric` modules respectively. In synthesizer these different objects are called "components".

Note that both a particle and parametric `Galaxy` can be intialised with any combination of `Stars`, `Gas`, or `BlackHoles`. Each is a keyword argument which all default to `None`.

### Global galaxy properties

In addition to the component attributes, a galaxy can also hold galaxy level attributes. These include a ``name`` for the galaxy, and more importantly the redshift of the galaxy, an attribute required to calculate the observer frame emission of the galaxy. Beyond the redshift, and like any other container object in synthesizer, the user can provide additional kwargs to the galaxy object, which will be stored as galaxy level attributes. This enables the storing of arbitrary data needed later in a pipeline (e.g. effective radii).

In [ ]:
from synthesizer import Galaxy

# Get a particle galaxy
part_gal = Galaxy(
    stars=part_stars,
    gas=None,
    black_holes=None,
    redshift=1,
    name="part_gal",
    gas_half_mass_radius=10 * kpc,
    stellar_half_mass_radius=4 * kpc,
)
print(part_gal)

print()

# Get a parametric galaxy
param_gal = Galaxy(
    stars=param_stars,
    redshift=1,
    name="param_gal",
    gas_half_mass_radius=6 * kpc,
    stellar_half_mass_radius=5 * kpc,
)
print(param_gal)

Note, that you are also free to avoid the factory function abstraction and explictly instantiate the desired `Galaxy` object directly.

In [ ]:
from synthesizer.particle import Galaxy

# Get a particle galaxy
part_gal = Galaxy(stars=part_stars, gas=None, black_holes=None, redshift=1)
print(part_gal)

Or for a parametric galaxy.

In [ ]:
from synthesizer.parametric import Galaxy

# Get a parametric galaxy
param_gal = Galaxy(stars=param_stars, redshift=1)
print(param_gal)